In [1]:
import regex as re
from functional import seq
from fn import _
from collections import namedtuple
from typing import List, Dict, Tuple, Optional
from termcolor import colored
import os

In [2]:
#Nr doesnt matter, position does

In [3]:
act_dir = os.environ["HOME"]+"/tmp/nlp/ustawy"
#act_file = "1993_599.txt"

In [4]:
act = open(act_dir + "/2003_592.txt").read() [20265:]
print(act)


[1])    Zmiany wymienionej ustawy zostały
ogłoszone w Dz. U. z 2000 r. Nr 120, poz. 1268, z 2002 r. Nr 25, poz. 253, z
2003 r. Nr 130, poz. 1188, z 2004 r. Nr 93, poz. 894, z 2005 r. Nr 172, poz.
1442 i Nr 267, poz. 2258, z 2006 r. Nr 157, poz. 1119, z 2007 r. Nr 176, poz.
1238 oraz z 2008 r. Nr 96, poz. 606.


[2])    Zmiany wymienionej ustawy zostały
ogłoszone w Dz. U. z 2003 r. Nr 217, poz. 2125, z 2004 r. Nr 91, poz. 870 i
871, Nr 96, poz. 959, Nr 121, poz. 1264, Nr 146, poz. 1546, Nr 173, poz. 1808 i
Nr 210, poz. 2135, z 2005 r. Nr 94, poz. 785, Nr 183, poz. 1538 i Nr 184, poz.
1539, z 2006 r. Nr 47, poz. 347, Nr 133, poz. 935 i Nr 157, poz. 1119, z 2007
r. Nr 123, poz. 850, Nr 179, poz. 1279 oraz z 2008 r. Nr 96, poz. 606.


[3])   Zmiany tekstu jednolitego wymienionej ustawy
zostały ogłoszone w Dz. U. z 2002 r. Nr 240, poz. 2055, z 2003 r. Nr 60, poz.
535 i Nr 90, poz. 844, z 2004 r. Nr 6, poz. 39, Nr 116, poz. 1207, Nr 123, poz.
1291, Nr 238, poz. 2390 i Nr 273, poz. 2703 i 27

In [8]:
def match_with_index(pattern ,text:str)-> List[Tuple[int,int,str]]:
    return [(m.start(0), m.end(0),m.captures()[0]) for m in re.finditer(pattern, text)]


def print_highlighted(
    text:str,
    matches_groups:List[List[Tuple[int,int]]],
    colors:List[Tuple[str,Optional[str]]]
):
    if len(matches_groups) != len(colors):
        raise "There should be the same number of matches groups and colors"
    if seq(matches_groups).exists(lambda x: len(x) == 0):
        print("sory bro, cannot do this"
             )
    
    
    MatchedEntry=namedtuple('MatchedEntry','beg end color on_color')
    
    # Seq[MatchedEntry]
    mg = seq(matches_groups)\
        .zip(colors)\
        .flat_map(lambda gc: seq(gc[0])\
            .map(lambda x : MatchedEntry(beg=x[0],end=x[1],color=gc[1][0], on_color = gc[1][1]))
                 )\
        .order_by(_.beg)
        
    # Will duplicate some matches but that's not the point
    beg = 0
    for matched in mg:
        print(text[beg:matched.beg],end="") 
        print(colored(text[matched.beg:matched.end],color= matched.color, on_color= matched.on_color),end="")
        beg = matched.end
    print(text[beg:])

In [9]:
dzu_pat = re.compile("Dz\.?\s?U.?")
year_pat = re.compile("((199)|(198)|(200)|(201))\d{1}")
nr_pat = re.compile("Nr\s+\d+") #TODO: word sep or sth
pos_pat = re.compile("poz\.\s+\d+(-\d+)*") #todo - "i \d"

#Should be ok, but needs extractors for entry
ustaw_pat=re.compile("(U|u)staw\w*\s+(z\s+dnia\s+)+\d+\s+\w+\s+(?P<year>\d{4})(\s|\n)*r\.?\s*(–|\-)*\s*(?P<title>(\w|\s|\")*)\((?P<entry>(.|\n)*?)\)")

#1) Ustawie ()
#(?<ala)  (?=ala)

In [ ]:
footer_pat = re.compile("\[\d+\]\)?\s+Zmiany(.|\s|\n)*?w\s+Dz\.*\s*U\.*(?P<entry>(.|\s|\n)*?)(?=(\[\d+\]))") #negative look ahead
matches = seq([
    #year_pat,
    dzu_pat,
    nr_pat,
    pos_pat,
    #ustaw_pat
    footer_pat
]).map(lambda pattern: match_with_index(pattern,act)).to_list()

print_highlighted(act,matches
                  ,[
                      #("green","on_grey"),
                      ("red","on_grey"),
                      ("blue","on_grey"),
                      ("yellow","on_grey"),
                      ("grey","on_yellow"),
                  ])

Find all external references to bills, e.g. **ustawie z dnia 4 marca 1994 r. o zakładowym funduszu świadczeń socjalnych (Dz. U. z 2012 r. poz. 592)**. The result should be aggregated by bill ID (year and position) and sorted by descending number of reference counts. The reference format should include:
        the title of the regulation (if present)
        the year of the regulation
        the number of the Journal of Laws of the Republic of Poland (Dziennik Ustaw) - if applicable
        the position of the regulation
